In [1]:
import metapy
import pandas as pd
import time
from numpy.random import shuffle

Load json dataset, save text and labels in separate files (line corpus format) - can be done only once

In [2]:
%%time
path=''
file='kindle_reviews.json'
df = pd.read_json(path_or_buf=path+file, lines=True, encoding='utf-8')    #, orient=None, typ='frame', dtype=True, convert_axes=True, convert_dates=True, keep_default_dates=True, numpy=False, precise_float=False, date_unit=None, encoding=None, chunksize=None, compression='infer')
print('Length of text: {}'.format(len(df)))

Length of text: 982619
Wall time: 12.1 s


In [3]:
df.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,B000F83SZQ,"[0, 0]",5,I enjoy vintage books and movies so I enjoyed ...,"05 5, 2014",A1F6404F1VG29J,Avidreader,Nice vintage story,1399248000
1,B000F83SZQ,"[2, 2]",4,This book is a reissue of an old one; the auth...,"01 6, 2014",AN0N05A9LIJEQ,critters,Different...,1388966400
2,B000F83SZQ,"[2, 2]",4,This was a fairly interesting read. It had ol...,"04 4, 2014",A795DMNCJILA6,dot,Oldie,1396569600
3,B000F83SZQ,"[1, 1]",5,I'd never read any of the Amy Brewster mysteri...,"02 19, 2014",A1FV0SX13TWVXQ,"Elaine H. Turley ""Montana Songbird""",I really liked it.,1392768000
4,B000F83SZQ,"[0, 1]",4,"If you like period pieces - clothing, lingo, y...","03 19, 2014",A3SPTOKDG7WBLN,Father Dowling Fan,Period Mystery,1395187200


In [4]:
df_text = df[['reviewText', 'overall']].copy()        # copy only certain columns to another df
#df = None
df_text['overall'] = df_text['overall'].apply(lambda x: 'pos' if x > 3 else 'neg' if x < 3 else 'mixed')

In [5]:
#df_text = df_text.sample(n=len(df_text))
df_text.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,pos
1,This book is a reissue of an old one; the auth...,pos
2,This was a fairly interesting read. It had ol...,pos
3,I'd never read any of the Amy Brewster mysteri...,pos
4,"If you like period pieces - clothing, lingo, y...",pos


In [6]:
df_text = df_text.loc[0:int(len(df_text)/50)]
print(len(df_text))
df_text['reviewText'].to_csv('ceeaus2/ceeaus2.dat', header=False, index=False)
df_text['overall'].to_csv('ceeaus2/ceeaus2.dat.labels', header=False, index=False)

19653


In [7]:
df_text.head()

,reviewText,overall
0,I enjoy vintage books and movies so I enjoyed ...,pos
1,This book is a reissue of an old one; the auth...,pos
2,This was a fairly interesting read. It had ol...,pos
3,I'd never read any of the Amy Brewster mysteri...,pos
4,"If you like period pieces - clothing, lingo, y...",pos


In [1]:
import metapy
import pandas as pd
import time

In [2]:
config = """stop-words = "lemur-stopwords.txt"

prefix = "."
dataset = "ceeaus2"
corpus = "line.toml"
index = "ceeaus2-idx"

[[analyzers]]
#method = "ngram-word"
#ngram = 1
#filter = "default-unigram-chain"

method = "ngram-word"
ngram = 1
    [[analyzers.filter]]
    type = "icu-tokenizer"
    
    [[analyzers.filter]]
    type = "lowercase"
    
    [[analyzers.filter]]
    type = "length"
    min = 1
    max = 35
    
    #[[analyzers.filter]]
    #type = "alpha"
    
    [[analyzers.filter]]
    type = "english-normalizer"    
"""
with open('ceeaus-config.toml', 'w') as f:
    f.write(config)

Save `ForwardIndex` to disk.

In [3]:
%%time
fidx = metapy.index.make_forward_index('ceeaus-config.toml')

Wall time: 1.09 s


The feature set used for classification depends on the settings in the configuration file _at the time of indexing_. Thus, if you change your `analyzer` pipeline (or other settings) - **reindex** your documents!

Decide what kind of dataset we're using - for binary classification (MeTA's `BinaryDataset`) or multi-class classification (`MulticlassDataset`). To see the number of labels:

In [4]:
fidx.num_labels()

3

Looks like we need a `MulticlassDataset` to predict which of these three labels a document should have (but if we are interested in one particular class only, we might use a `BinaryDataset`).

For now, let's focus on the multi-class case, as that likely makes the most sense for this kind of data. Since the dataset is small enough, we can load all documents into memory at once like this.

In [5]:
dset = metapy.classify.MulticlassDataset(fidx)
len(dset)

19653

Labels

In [6]:
set([dset.label(instance) for instance in dset])

{'mixed', 'neg', 'pos'}

Since datasets may be large, it's beneficial to avoid creating copies of them (e.g. to shuffle them) => you can operate with a `DatasetView` (`MulticlassDatasetView` or `BinaryDatasetView`) to shuffle or rotate the dataset without modifying it - use Python's slicing (will it really avoid a modification of both datasets?) or construct a view directly.

In [7]:
#view = dset[0:len(dset)+1]
# or
view = metapy.classify.MulticlassDatasetView(dset)

Shuffle the view without changing the underlying datsaet.

In [8]:
view.shuffle()
print("{} vs {}".format(view[0].id, dset[0].id))

13247 vs 0


The view is shuffled in random order, but the underlying dataset is still sorted by id.

Slice the shuffled view for a train and test data split.

In [9]:
training = view[0:int(0.8*len(view))]
testing = view[int(0.8*len(view)):len(view)+1]

Train a Naive Bayes classifier on the training view.

In [10]:
%%time
nb = metapy.classify.NaiveBayes(training)

Wall time: 281 ms


Classify individual documents

In [11]:
# classify individual documents
#nb.classify(testing[0].weights)

Classify the test set

In [12]:
mtrx = nb.test(testing)
print(mtrx)


           mixed    neg      pos      
         ---------------------------
   mixed | 0.373    0.213    0.414    
     neg | 0.239    0.532    0.229    
     pos | 0.119    0.0541   0.827    




`Test()` method returns a `ConfusionMatrix` which computes a lot of metrics used in classifier evaluation. Shuffling may lead to different results. **Each row says what fraction of documents with that _true_ label were assigned to other labels**.

In [13]:
mtrx.print_stats()

------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
mixed       0.348       0.326       0.373       0.148       
neg         0.479       0.435       0.532       0.104       
pos         0.852       0.879       0.827       0.748       
------------------------------------------------------------
Total       0.74        0.751       0.729       
------------------------------------------------------------
3931 predictions attempted, overall accuracy: 0.729



[Cross-validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) - to avoid overfitting; running CV across the whole dataset to get an idea of how well we might generalize to new data.

In [14]:
mtrx = metapy.classify.cross_validate(lambda fold: metapy.classify.NaiveBayes(fold), view, 5)

`Cross_validate()` also returns a `ConfusionMatrix`; arguments - function to create the trained classifiers for each fold, a dataset view (all documents), and the number of folds.

In [15]:
print(mtrx)
mtrx.print_stats()


           mixed    neg      pos      
         ---------------------------
   mixed | 0.358    0.205    0.437    
     neg | 0.236    0.548    0.215    
     pos | 0.119    0.0592   0.822    


------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
mixed       0.33        0.305       0.358       0.14        
neg         0.482       0.43        0.548       0.101       
pos         0.851       0.883       0.822       0.759       
------------------------------------------------------------
Total       0.743       0.756       0.73        
------------------------------------------------------------
19650 predictions attempted, overall accuracy: 0.73



Now the same for [SVM](https://en.wikipedia.org/wiki/Support_vector_machine).

MeTA's implementation of SVM is an approximation using [stochastic gradient descent](https://en.wikipedia.org/wiki/Stochastic_gradient_descent) on the [hinge loss](https://en.wikipedia.org/wiki/Hinge_loss) implemented as a `BinaryClassifier`. In case of multi-class clasification - use adapters: [One-vs-All](https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-rest) and [One-vs-One](https://en.wikipedia.org/wiki/Multiclass_classification#One-vs.-one). Constructing `OneVsAll` reduction by passing the dataset, binary classifier, and its arguments:

In [16]:
ova = metapy.classify.OneVsAll(training, metapy.classify.SGD, loss_id='hinge')

Then use `OneVsAll` as a classifier.

In [17]:
mtrx = ova.test(testing)
print(mtrx)
mtrx.print_stats()


           mixed    neg      pos      
         ---------------------------
   mixed | 0.0704   0.0962   0.833    
     neg | 0.0439   0.317    0.639    
     pos | 0.0109   0.00749  0.982    


------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
mixed       0.122       0.451       0.0704      0.148       
neg         0.421       0.625       0.317       0.104       
pos         0.878       0.794       0.982       0.748       
------------------------------------------------------------
Total       0.751       0.726       0.777       
------------------------------------------------------------
3931 predictions attempted, overall accuracy: 0.777



In [18]:
mtrx = metapy.classify.cross_validate(lambda fold: metapy.classify.OneVsAll(fold, metapy.classify.SGD, loss_id='hinge'), view, 5)
print(mtrx)
mtrx.print_stats()


           mixed    neg      pos      
         ---------------------------
   mixed | 0.243    0.141    0.616    
     neg | 0.178    0.463    0.36     
     pos | 0.0517   0.0201   0.928    


------------------------------------------------------------
Class       F1 Score    Precision   Recall      Class Dist  
------------------------------------------------------------
mixed       0.294       0.372       0.243       0.14        
neg         0.512       0.572       0.463       0.101       
pos         0.888       0.852       0.928       0.759       
------------------------------------------------------------
Total       0.771       0.757       0.785       
------------------------------------------------------------
19650 predictions attempted, overall accuracy: 0.785



To see a list of what's included in the bindings:

In [19]:
help(metapy.classify)

Help on module metapy.metapy.classify in metapy.metapy:

NAME
    metapy.metapy.classify

SUBMODULES
    kernel

CLASSES
    metapy.metapy.learn.Dataset(pybind11_builtins.pybind11_object_48)
        BinaryDataset
        MulticlassDataset
    metapy.metapy.learn.DatasetView(pybind11_builtins.pybind11_object_48)
        BinaryDatasetView
        MulticlassDatasetView
    pybind11_builtins.pybind11_object_48(builtins.object)
        BinaryClassifier
            OnlineBinaryClassifier
                SGD
        Classifier
            DualPerceptron
            KNN
            LogisticRegression
            NaiveBayes
            NearestCentroid
            OnlineClassifier
                OneVsAll
                OneVsOne
            Winnow
        ConfusionMatrix
        Kernel
    
    class BinaryClassifier(pybind11_builtins.pybind11_object_48)
     |  Method resolution order:
     |      BinaryClassifier
     |      pybind11_builtins.pybind11_object_48
     |      builtins.object
   

Another way to create config.toml

In [ ]:
config2 = """stop-words = "lemur-stopwords.txt"

[[analyzers]]
method = "ngram-word"
ngram = 1
    [[analyzers.filter]]
    type = "icu-tokenizer"

    [[analyzers.filter]]
    type = "lowercase"

    [[analyzers.filter]]
    type = "length"
    min = 2
    max = 35

[[analyzers]]
method = "ngram-pos"
ngram = 2
filter = [{type = "icu-tokenizer"}, {type = "ptb-normalizer"}]
crf-prefix = "crf"

[[analyzers]]
method = "tree"
filter = [{type = "icu-tokenizer"}, {type = "ptb-normalizer"}]
features = ["subtree"]
tagger = "perceptron-tagger/"
parser = "parser/"
"""
#with open('config.toml', 'w') as f:
#    f.write(config2)

In [ ]:
config3 = """prefix = "."
dataset = "ceeaus"
corpus = "line.toml"
index = "ceeaus-idx"
stop-words = "lemur-stopwords.txt"

[[analyzers]]
method = "ngram-word"
ngram = 1
filter = "default-unigram-chain"
"""
#with open('ceeaus-config.toml', 'w') as f:
#    f.write(config3)